In [1]:
import os
from os import path
import sys

import json
import re
import torch

import transformers
from sentence_transformers import SentenceTransformer

from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http import models
from qdrant_client.http.models import CollectionStatus
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, FieldCondition, MatchValue, Filter

# qdrant = QdrantClient(url="http://qdrant:6333")
qdrant = QdrantClient(host="172.18.0.4", port=6333)

model = SentenceTransformer("all-MiniLM-L6-v2")


qdrant_collection_name = "ies_website_embeddings_msmarco_german" 

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"using {device} as device")

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:1614: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Device 0: NVIDIA RTX A5000
using cuda as device


In [2]:
print("max sequence length of sentence transformer:", model.max_seq_length)

max sequence length of sentence transformer: 256


In [3]:
# emb = model.encode("Wir entwickeln Softwarewerkzeuge für das regulatorische Monitoring.")
emb = model.encode("Wir setzen auf aktuelle und etablierte Technologien um Ihnen eine robuste Basis für Ihre digitalisierte Zukunft zu bieten.")
hits = qdrant.search(
    collection_name="ies_website_embeddings",
    query_vector = emb,
    limit=10
)

for hit in hits:
    print(hit.payload["name"], hit.payload["text"])
    print("------------")

in4s GmbH Wir setzen auf aktuelle und etablierte Technologien um Ihnen eine robuste Basis für Ihre digitalisierte Zukunft zu bieten.
------------
Handwerkskammer Halle (Saale) Aber oft stellt die Digitalisierung Betriebe vor große Herausforderungen, da viele Technologien, Möglichkeiten und Zusammenhänge nicht bekannt oder erkannt werden.
------------
E+Service+Check GmbH Auch die Digitalisierung kann hierbei helfen, indem Prüfungen elektronisch erfasst und dokumentiert werden.
------------
Hansestadt Gardelegen Im Förderkompass des Zukunftszentrums Digitale Arbeit finden Sie Unterstützungsangebote für Digitalisierung und Strukturwandel.
------------
Trianel Solarpark Uchtdorf GmbH &amp; Co. KG Die Digitalisierung bietet enorme Chancen, Prozesse mit hohem manuellem Aufwand zu vereinfachen, zu beschleunigen und Fehler zu...
------------
Trianel Solarpark Uchtdorf GmbH &amp; Co. KG Die Digitalisierung bietet enorme Chancen, Prozesse mit hohem manuellem Aufwand zu vereinfachen, zu beschleu

In [5]:
qdrant.search(
    collection_name="ies_website_embeddings",    
    limit=100,
    query_vector=[0.0]*len(emb),
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="name",
                match=models.MatchValue(value="regiocom SE")  # Name you want to filter by
            )
        ]
    )
)

ResponseHandlingException: timed out

In [9]:
# emb = model.encode("Wir entwickeln Softwarewerkzeuge für das regulatorische Monitoring.")
emb = model.encode("In unserem Hause werden Knochen gerichtet und Knochenbrüche geheilt.")
print(len(emb))
hits = qdrant.search(
    collection_name="ies_website_embeddings",
    query_vector = emb,
    limit=10
)

for hit in hits:
    print(hit.payload["name"], hit.payload["text"])
    print("------------")

384


In [ ]:
# Define the query
query = {
    "must": [
        {"key": "crefoid", "match": {"value": "133476"}},
        {"key": "filepath", "match": {"value": "/local/innecs-local/website-data/3130446512_scorprotect-de/isolierung-daemmung-rohrisolierung-alukaschiert.txt"}},
        {"key": "idx", "match": {"value": 80}}
    ]
}

# Retrieve the documents
response = qdrant.search(collection_name="ies_website_embeddings", query_vector=[0.0]*len(emb), query=query)

# Print the retrieved document
for hit in response:
    print(hit)


In [ ]:
from qdrant_client.models import Distance, VectorParams, FieldCondition, MatchValue, Filter

qdrant.search(
    collection_name="ies_website_embeddings",    
    limit=1,
    query_vector=[0.0]*len(emb),
    query_filter = Filter(
    must=[
        FieldCondition(
            key="crefoid",
            match=MatchValue(value="133476")
        ),
        FieldCondition(
            key="filepath",
            match=MatchValue(value="/local/innecs-local/website-data/3130446512_scorprotect-de/isolierung-daemmung-rohrisolierung-alukaschiert.txt")
        ),
        FieldCondition(
            key="idx",
            match=MatchValue(value=80)
        )
    ]
)
)